In [1]:
import os

# Set the proxy environment variables
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'

In [2]:
import ee
import datetime
import itertools
import sys
import re

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [3]:
ee.Initialize()

In [4]:
# define basic parameters
year_span = [f'{i}_{i+2}' for i in range(2020,2023,3)]

In [5]:
# define the region to analysis
region    = ['华东',   '东北',   '中南',    '华北',  '西北', '西南']
region_en = ['huadong','dongbei','zhongnan','huabei','xibei','xinan']

In [42]:
region_cn = '西北'
region_en = 'xibei'

In [50]:
# import COPERNICUS.urban_coverfraction layer, which will be used to masking urban classfications
COPERNICUS_urban = ee.Image(f'COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019').select("urban-coverfraction")

# other masking layers
dem = ee.Image("USGS/SRTMGL1_003")
slope = ee.Terrain.slope(dem)
precipitation = ee.Image("users/wangjinzhulala/China_built_up/07_supplementary_data/China_precipication_2010s")


# create the mask_rule_dict
mask_rule = {'dongbei':precipitation.lt(550),
            'huabei':slope.gt(4),
            'huadong':slope.gt(4),
            'xibei':dem.gt(1000).Or(precipitation.lt(500)).Or(slope.gt(4)),
            'xinan':dem.gt(3000),
            'zhongnan':slope.gt(4)}

# function to perform the masking
def mask_img(in_img,region_name=region_en):
    
    # 1) first clip out the area to be masked by COPERNICUS_urban
    img_mask_area = in_img.updateMask(mask_rule[region_name])
    img_mask_area = img_mask_area.multiply(COPERNICUS_urban.gt(20))
    
    # 2) then clip out the area remain unchanged
    img_unchange_area = in_img.updateMask(mask_rule[region_name].Not())
    
    # 3) mosaic the two imgs
    merge_img = ee.ImageCollection([img_mask_area,img_unchange_area]).mosaic()
    
    
    return merge_img

##### Get the priliminary classifications

In [51]:
# prepare the different input band types
in_band_selection =[#'Spectrum',
                    #'Spectrum_Normalize',
                    #'Spectrum_Normalize_Fourier',
                    #'Spectrum_Normalize_Fourier_Terrain',
                    'Spectrum_Normalize_Fourier_Terrain_Meterology']

In [52]:
# get the priliminary classifications and select those with a 
# >5 value in the 10-folds sum, here the 5 threshold comes from
# the Step_1 10-folds check of this fold

path = 'users/wangjinzhulala/China_built_up/05_primary_classification'

classification_ten_folds_check = {}

for in_band in in_band_selection:
    for year in year_span:
        
        img_name = f'{in_band}_{region_en}_{year}'
    
        preliminary_classification = ee.ImageCollection([ee.Image(f'{path}/{img_name}_{seed}_zhongnan_sample') for seed in range(10)]).sum()
        img_check = preliminary_classification.gt(4)
        
        # masking: from the classfication tricks
        if int(year[:4]) > 2013:
            # !!!! change mask_img accordingly !!!!
            img_check = mask_img(img_check)

        classification_ten_folds_check[(in_band,year)] = [img_check]
        

In [53]:
# store the classification in a df for conviniency
classification_ten_folds_check_df = pd.DataFrame(classification_ten_folds_check)

# format the df
classification_checked = classification_ten_folds_check_df.T.reset_index()
classification_checked.columns = ['In bands','year','classification']

In [54]:
classification_checked

,In bands,year,classification
0,Spectrum_Normalize_Fourier_Terrain_Meterology,2020_2022,"ee.Image({\n ""functionInvocationValue"": {\n ..."


##### Export the result

In [55]:
# import north_china_plain boundary
Region =  ee.FeatureCollection("users/wangjinzhulala/China_built_up/01_Boundary_shp/China_zone")\
            .filterMetadata('NAME1','equals',region_cn)

In [56]:
one_img = classification_checked.iloc[0]['classification']

# add the selected classification in map
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.centerObject(Region,6)
Map.addLayer(one_img,{'min':0,'max':1},'One')
Map

Map(center=[39.17019792517029, 92.01646570060687], controls=(WidgetControl(options=['position', 'transparent_b…

In [57]:
for idx,row in classification_checked.iterrows():
    
    out_path = 'users/wangjinzhulala/China_built_up/05_primary_classification'
    
    # get data
    in_bands = row[0]
    year = row[1]
    img = row[2]
    
    # construct export name
    name = f'{in_bands}_{region_en}_{year}_ten_fold_check_huadong_sample'
    
    # export
    task = ee.batch.Export.image.toAsset(    
                                            image= img,
                                            description=name,
                                            assetId=f'{out_path}/{name}',
                                            region=Region.geometry().bounds(),
                                            scale=30,
                                            maxPixels=int(1e13)
    )
    
    task.start()
    
    # print out the process
    print(f'{name} has been exported!')

Spectrum_Normalize_Fourier_Terrain_Meterology_xibei_2020_2022_ten_fold_check_huadong_sample has been exported!
